In [106]:
import pandas as pd
import re
import regex as re
import os
#sys.path.append('../')
#from utils import find_lines_with_player, name_wordgroups, remove_similar_rows_per_player
import numpy as np
from gensim.parsing.preprocessing import strip_punctuation

In [107]:
#in utils. for joana
from difflib import SequenceMatcher

# Function which finds the lines where a players name is contained
def find_lines_with_player(dataframe, playerlist, n_lines = 0):
    
    # create empty df 
    df_complete = pd.DataFrame()

    # iterating over all players
    for player in playerlist:

        # get players first and last_name to include them in later sentence checks
        player_first_name, player_last_name = player.split()

        # just select player indiviual data
        df_player = dataframe[dataframe["player"] == player]
        df_player = df_player.reset_index(drop=True)

        # iterate over all data for the player
        for i in range(len(df_player)):

            # get the current record
            current_line = df_player['data'].iloc[i]
            # split up the records in lines
            lines = current_line.split('\\n')
            # create an empty string
            new_string = ''

            line_counter = 0
            # iterate over all lines in the record
            for line in lines:
                # if the playername can be found in the line add the line to the string
                if line.find(player) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
            
                elif line.find(player_first_name) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
        
                elif line.find(player_last_name) != -1:
                    new_string = new_string + line + " "
                    if line_counter <= 0:
                        line_counter = line_counter + n_lines
            
                elif line_counter >= 0:
                    new_string = new_string + line + " "
                    line_counter = line_counter-1
        
            # switch the previos record against the newly created string 
            df_player['data'].iloc[i] = new_string

        # add the new data to the Dataframe and return
        df_complete = pd.concat([df_complete, df_player], axis=0)
        
    return df_complete

def remove_similar_rows_per_player(df, playerlist, threshold=0.9):
    '''The procedure of deleting similiar articles needs to be done by each player because if an article writes about 
    # e.g. two players we want to keep it for both of the players'''

    # define empty df which will be returned in the end
    df_complete = pd.DataFrame()

    for player in playerlist:
        
        # create the df for the player
        df_player = df[df["player"] == player]
        df_player = df_player.reset_index(drop=True)
        column_as_df = pd.DataFrame(df_player['data'])


        
        # Compute similarity scores for each pair of rows
        similarity_scores = {}
        for i, row in column_as_df.iterrows():
            for j, other_row in column_as_df.iterrows():
                if i >= j:
                    continue
                score = SequenceMatcher(None, row, other_row).ratio()
                if score >= threshold:
                    similarity_scores[(i, j)] = score
        
        # Identify rows to remove
        rows_to_remove = []
        for (i, j), score in similarity_scores.items():
            if i not in rows_to_remove and j not in rows_to_remove:
                rows_to_remove.append(j if df_player.index[i] < df_player.index[j] else i)
        
        # Remove rows and concatenate df
        df_player = df_player.drop(rows_to_remove)
        df_complete = pd.concat([df_complete, df_player], axis=0)

        #return modified DataFrame
    return df_complete

'''def name_wordgroups(df):
    #    Function to match first and surname to just last name
    # create patterns which should be matched 
    # first lastname and firstname should both result in just lastname
    pattern_match2d = np.array([[r"\b(mitchel bakker|mitchel)\b", 'bakker'], 
                                [r"\b(xabi alonso|xabi)\b", 'alonso'], 
                                [r"\b(exequiel palacios|exequiel)\b", 'palacios'],
                                [r"\b(nadiem amiri|nadiem)\b", 'amiri'],
                                [r"\b(kerem demirbay|kerem)\b", 'demirbay'],
                                [r"\b(robert andrich|robert)\b", 'andrich'],
                                [r"\b(exequiel palacios|exequiel)\b", 'palacios'],
                                [r"\b(piero hincapie|piero)\b", 'hincapie'],
                                [r"\b(jeremie frimpong|jeremie)\b", 'frimpong'],
                                [r"\b(jonathan tah|jonathan)\b", 'tah'],
                                [r"\b(moussa diaby|moussa)\b", 'diaby'],
                                [r"\b(mykhaylo mudryk|mykhaylo)\b", 'mudryk'],
                                [r"\b(amine adli|amine)\b", 'adli'],
                                [r"\b(florian wirtz|florian)\b", 'wirtz'],
                                [r"\b(jose mourinho|jose)\b", 'mourinho'],     
                                #other wordgroups
                                [r"\b(europa league)\b", 'europaleague'],
                                [r"\b(champions league)\b", 'championsleague'],
                                [r"\b(bayer leverkusen|bayer|leverkusen|leverkusens)\b", 'bayerleverkusen']
                                ])

    # do the pattern matching for each player
    for pattern, player in pattern_match2d:
        df['data'] = df['data'].apply(lambda x: re.sub(pattern, str(player), str(x)))

    return df
'''

'def name_wordgroups(df):\n    #    Function to match first and surname to just last name\n    # create patterns which should be matched \n    # first lastname and firstname should both result in just lastname\n    pattern_match2d = np.array([[r"\x08(mitchel bakker|mitchel)\x08", \'bakker\'], \n                                [r"\x08(xabi alonso|xabi)\x08", \'alonso\'], \n                                [r"\x08(exequiel palacios|exequiel)\x08", \'palacios\'],\n                                [r"\x08(nadiem amiri|nadiem)\x08", \'amiri\'],\n                                [r"\x08(kerem demirbay|kerem)\x08", \'demirbay\'],\n                                [r"\x08(robert andrich|robert)\x08", \'andrich\'],\n                                [r"\x08(exequiel palacios|exequiel)\x08", \'palacios\'],\n                                [r"\x08(piero hincapie|piero)\x08", \'hincapie\'],\n                                [r"\x08(jeremie frimpong|jeremie)\x08", \'frimpong\'],\n                         

In [108]:
'''
def extract_sentence(df):
    player_info_sentences = []
    for idx, row in df.iterrows():
        player = row['player']
        entry = row['data']
        sentences = re.split('\. ', entry)
        found_sentence = False
        for sentence in sentences:
            if player in sentence:
                player_info_sentences.append(sentence)
                found_sentence = True
                break
        if not found_sentence:
            player_info_sentences.append('')
    
    df['sentence'] = player_info_sentences
    return df
'''


def extract_sentence(df):
    player_info_sentences = []
    for idx, row in df.iterrows():
        player = row['player']
        data = row['data']  # Updated column name from 'entry' to 'data'
        sentences = re.split('\.\s*', data)
        found_sentence = False
        for sentence in sentences:
            if player.lower() in sentence.lower():  # Perform case-insensitive comparison
                player_info_sentences.append(sentence)
                found_sentence = True
                break
        if not found_sentence:
            player_info_sentences.append('')

    df['sentence'] = player_info_sentences
    return df



In [109]:
# Load the CSV file into a DataFrame
df_de = pd.read_csv('../Preprocessing/data_clean/de_clean_1.csv')
df_en = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/en_clean_1.csv')
df_en_con = pd.read_csv('https://raw.githubusercontent.com/svisel22/SS23-BIPM-Analytics-Lab---Group-4-repository/main/Preprocessing/data_clean/en_clean_condensed_punc_play.csv')
df_es = pd.read_csv('../Preprocessing/data_clean/es_clean_1.csv')

In [110]:
# Keep paragraph where the player name is found
df_de = find_lines_with_player(df_de, df_de['player'].unique(),n_lines=1)
df_en = find_lines_with_player(df_en, df_en['player'].unique(),n_lines=1)
df_en_con = find_lines_with_player(df_en_con, df_en_con['player'].unique(),n_lines=1)
df_es = find_lines_with_player(df_es, df_es['player'].unique(),n_lines=1)

In [111]:
# Extract sentence
df_de = extract_sentence(df_de)
df_en = extract_sentence(df_en)
df_en_con = extract_sentence(df_en_con)
df_es = extract_sentence(df_es)

In [112]:
df_de

,data,player,language,publishedAt,sentence
0,trainer xabi alonso vor den mitgereisten fans ...,exequiel palacios,de,2023-02-24T09:33:31Z,trainer xabi alonso vor den mitgereisten fans ...
1,sechser droht nach viereinhalb monaten pause b...,exequiel palacios,de,2023-03-03T21:35:13Z,zudem ist die konkurrenzsituation auf der dopp...
2,leverkusens sechser erklart vor dem budapest-s...,exequiel palacios,de,2023-03-07T11:34:39Z,wie auch exequiel palacios sah der defensive m...
3,mitchel bakker am mittwoch im abschluss-traini...,exequiel palacios,de,2023-03-08T14:25:18Z,",,er ist eine option"", erklart alonso, der im ..."
4,allerdings waren in robert andrich und dem ...,exequiel palacios,de,2023-03-09T19:53:46Z,allerdings waren in robert andrich und dem ...
...,...,...,...,...,...
80,bayer leverkusen wehrt sich in der europa leag...,piero hincapie,de,2023-05-12T10:02:12Z,"sekunden waren gespielt, als robert andrich na..."
81,xabi alonso hat im taktik-duell mit seinem alt...,piero hincapie,de,2023-05-11T21:17:00Z,"sekunden waren gespielt, als robert andrich na..."
82,trotz der pleite im halbfinal-hinspiel bei der...,piero hincapie,de,2023-05-11T21:09:16Z,"sekunden waren gespielt, als robert andrich na..."
83,nach dem hitzigen aufeinandertreffen im stadio...,piero hincapie,de,2023-05-11T20:58:58Z,"sekunden waren gespielt, als robert andrich na..."


In [113]:
df_en

,data,player,language,publishedAt,sentence
0,"monaco, feb (reuters) - bayer leverkusen beat ...",exequiel palacios,en,2023-02-23T20:50:50Z,leverkusen took the lead again in the st minut...
1,the star edition change location this copy is ...,exequiel palacios,en,2023-03-03T16:42:19Z,"midfielders: leandro paredes (juventus), angel..."
2,bayer leverkusen became the third side to beat...,exequiel palacios,en,2023-03-19T18:30:00Z,a first half goal from joshua kimmich was canc...
3,"""by reuters bayer leverkusen's exequiel palac...",exequiel palacios,en,2023-03-19T18:42:59Z,"""by reuters bayer leverkusen's exequiel palac..."
4,bayer leverkusen\'s exequiel palacios scored t...,exequiel palacios,en,2023-03-19T19:05:09Z,bayer leverkusen\'s exequiel palacios scored t...
...,...,...,...,...,...
16,the hammers have struggled defensively this s...,piero hincapie,en,2023-05-22T12:32:54Z,
17,hello and welcome to another edition of our t...,piero hincapie,en,2023-05-22T11:40:14Z,manchester united's aaron wan-bissaka is one d...
18,everyone seemed to have worked through the sa...,piero hincapie,en,2023-05-21T17:00:46Z,kendry paez is the latest exciting young playe...
19,tottenham have identified form bayer leverkus...,piero hincapie,en,2023-05-16T12:22:18Z,steidten is credited as having played a key ro...


In [114]:
df_en_con

,data,player,language,publishedAt,sentence
0,"monaco, feb (reuters) bayerleverkusen beat mon...",exequiel palacios,en,2023-02-23T20:50:50Z,
1,star edition change location copy personal non...,exequiel palacios,en,2023-03-03T16:42:19Z,
2,bayerleverkusen beat bayern munich bundesliga ...,exequiel palacios,en,2023-03-19T18:30:00Z,
3,"""by reuters bayerleverkusen's scored second-h...",exequiel palacios,en,2023-03-19T18:42:59Z,
4,bayerleverkusen\'s scored second-half penalti...,exequiel palacios,en,2023-03-19T19:05:09Z,
...,...,...,...,...,...
16,"hammers struggled defensively season, shipping...",piero hincapie,en,2023-05-22T12:32:54Z,
17,"hello welcome edition transfer newsletter, bro...",piero hincapie,en,2023-05-22T11:40:14Z,
18,"worked inner turmoil. thought going. mean, poi...",piero hincapie,en,2023-05-21T17:00:46Z,
19,tottenham identified form bayerleverkusen werd...,piero hincapie,en,2023-05-16T12:22:18Z,


In [115]:
df_es

,data,player,language,publishedAt,sentence
0,adeyemi firmo el que es su primer gol en lo qu...,exequiel palacios,es,2023-01-29T18:25:03Z,adeyemi firmo el que es su primer gol en lo qu...
1,la ultima semana de marzo es la primera ventan...,exequiel palacios,es,2023-01-30T16:52:46Z,
2,gustavo puerta ya no jugara en el bayer leverk...,exequiel palacios,es,2023-01-31T20:41:38Z,"el club aleman, que siempre se ha caracterizad..."
3,"alberto fernandez el presidente de la afa, cl...",exequiel palacios,es,2023-02-09T18:32:38Z,"alberto fernandez el presidente de la afa, cl..."
4,alberto fernandez tambien participaron los ot...,exequiel palacios,es,2023-02-12T21:13:55Z,alberto fernandez tambien participaron los ot...
...,...,...,...,...,...
68,"antes del minuto, el visitante ya habia llegad...",piero hincapié,es,2023-05-11T21:13:48Z,
69,el bayer leverkusen jugo de visita contra la r...,piero hincapié,es,2023-05-11T20:56:21Z,
70,un ecuatoriano volvera a semifinales de un tor...,piero hincapié,es,2023-05-11T18:30:25Z,
71,la seleccion de ecuador jugara sus dos ultimos...,piero hincapié,es,2023-05-10T23:37:55Z,


In [40]:
df_en_con.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
 4   sentence     363 non-null    object
dtypes: object(5)
memory usage: 17.0+ KB


In [41]:
# copy sentence column into data and remove sentence
df_de['data']= df_de['sentence']
df_de.drop('sentence', axis=1, inplace=True)
df_en['data']= df_en['sentence']
df_en.drop('sentence', axis=1, inplace=True)
df_en_con['data']= df_en_con['sentence']
df_en_con.drop('sentence', axis=1, inplace=True)
df_es['data']= df_es['sentence']
df_es.drop('sentence', axis=1, inplace=True)

In [42]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


In [43]:
df_en_con['data'] = df_en_con['data'].apply(strip_punctuation)
df_en_con.head()

,data,player,language,publishedAt
0,,exequiel palacios,en,2023-02-23T20:50:50Z
1,,exequiel palacios,en,2023-03-03T16:42:19Z
2,,exequiel palacios,en,2023-03-19T18:30:00Z
3,,exequiel palacios,en,2023-03-19T18:42:59Z
4,,exequiel palacios,en,2023-03-19T19:05:09Z


In [44]:
df_en_con.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         363 non-null    object
 1   player       363 non-null    object
 2   language     363 non-null    object
 3   publishedAt  363 non-null    object
dtypes: object(4)
memory usage: 14.2+ KB


In [45]:
df_en_con

,data,player,language,publishedAt
0,,exequiel palacios,en,2023-02-23T20:50:50Z
1,,exequiel palacios,en,2023-03-03T16:42:19Z
2,,exequiel palacios,en,2023-03-19T18:30:00Z
3,,exequiel palacios,en,2023-03-19T18:42:59Z
4,,exequiel palacios,en,2023-03-19T19:05:09Z
...,...,...,...,...
16,,piero hincapie,en,2023-05-22T12:32:54Z
17,,piero hincapie,en,2023-05-22T11:40:14Z
18,,piero hincapie,en,2023-05-21T17:00:46Z
19,,piero hincapie,en,2023-05-16T12:22:18Z


In [17]:
# Remove the similiar rows (The Function is imported from utils on top)
df_de = remove_similar_rows_per_player(df_de, df_de['player'].unique())
df_en = remove_similar_rows_per_player(df_en, df_en['player'].unique())
df_en_con = remove_similar_rows_per_player(df_en_con, df_en_con['player'].unique())
df_es = remove_similar_rows_per_player(df_es, df_es['player'].unique())

In [18]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         143 non-null    object
 1   player       143 non-null    object
 2   language     143 non-null    object
 3   publishedAt  143 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [19]:
df_en_con.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 0
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         7 non-null      object
 1   player       7 non-null      object
 2   language     7 non-null      object
 3   publishedAt  7 non-null      object
dtypes: object(4)
memory usage: 280.0+ bytes


In [20]:
''' #performed in preprocessing - at least for english
# Change names in the text to unify player name to last name
df_de = name_wordgroups(df_de)
df_en = name_wordgroups(df_en)
df_en_con = name_wordgroups(df_en_con)
df_es = name_wordgroups(df_es)'''

In [21]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 143 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         143 non-null    object
 1   player       143 non-null    object
 2   language     143 non-null    object
 3   publishedAt  143 non-null    object
dtypes: object(4)
memory usage: 5.6+ KB


In [22]:
df_en_con.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7 entries, 0 to 0
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         7 non-null      object
 1   player       7 non-null      object
 2   language     7 non-null      object
 3   publishedAt  7 non-null      object
dtypes: object(4)
memory usage: 280.0+ bytes


In [23]:
# delete empty rows
df_de = df_de.replace('', pd.NA)
df_de.dropna(inplace=True)

df_en = df_en.replace('', pd.NA)
df_en.dropna(inplace=True)

df_en_con = df_en_con.replace('', pd.NA)
df_en_con.dropna(inplace=True)

df_es = df_es.replace('', pd.NA)
df_es.dropna(inplace=True)

In [24]:
df_en.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         136 non-null    object
 1   player       136 non-null    object
 2   language     136 non-null    object
 3   publishedAt  136 non-null    object
dtypes: object(4)
memory usage: 5.3+ KB


In [25]:
df_en_con.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   data         0 non-null      object
 1   player       0 non-null      object
 2   language     0 non-null      object
 3   publishedAt  0 non-null      object
dtypes: object(4)
memory usage: 0.0+ bytes


In [26]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'en_clean_1_sen.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_en.to_csv(file_path, index=False)

In [27]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'en_clean_con_sen.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_en_con.to_csv(file_path, index=False)

In [30]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'de_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_de.to_csv(file_path, index=False)

In [32]:
# Create the folder if it doesn't exist
folder_name = 'data_clean'

# Define the file path for saving the CSV
file_name = 'es_clean_1-1.csv'
file_path = os.path.join(folder_name, file_name)

# Convert the dataframe to CSV and save it
df_es.to_csv(file_path, index=False)